# import data

In [1]:
import pandas as pd
import numpy as np
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

22.11.1.52440
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [2]:
df_origin = pd.read_excel('stocks.xlsx')
df_contract_value =  df_origin.set_index('date')[['lastUpdateTime','Unnamed: 82']].reset_index()
df_contract_value.columns = ['code','value','industry']
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_wind =  w.wss(f"{stk_code_str}", "sec_name", "tradeDate=20221102;",usedf=True)[1]
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_info = df_contract_value.set_index('code').loc[stk_code_list]
# pd.set_option('display.max_rows', None)
df_stk_info['name'] = df_stk_wind['SEC_NAME']
df_stk_clean = df_stk_info[df_stk_info['industry']!=0].fillna('工业').sort_values(by='industry').reset_index()
df_stk_clean = df_stk_clean[['code','name','value','industry']]
# pd.set_option('display.max_rows', None)
df_stk_clean['value'] = df_stk_clean['value'].apply(lambda x: format(x, 'f'))

tmp_str = ','.join(df_stk_clean['code'].tolist())
df_ipo_date = w.wss(tmp_str, "ipo_date", "",usedf=True)[1].reset_index()
df_stk_clean['IPO_DATE'] = df_ipo_date['IPO_DATE']
# df_stk_clean.to_excel('stock_hold.xlsx')

218
218


In [3]:
# pd.set_option('display.max_rows', None)
df_stk_clean

,code,name,value,industry,IPO_DATE
0,601006.SH,大秦铁路,1675401.000000,交通运输,2006-08-01
1,000088.SZ,盐田港,127506.000000,交通运输,1997-07-28
2,600798.SH,宁波海运,1770720.000000,交通运输,1997-04-23
3,000099.SZ,中信海直,500916.000000,交通运输,2000-07-31
4,002027.SZ,分众传媒,7490642.000000,传媒,2004-08-04
...,...,...,...,...,...
208,002726.SZ,龙大美食,20577789.540000,食品饮料,2014-06-26
209,002626.SZ,金达威,10064392.000000,食品饮料,2011-10-28
210,000858.SZ,五粮液,1425840.000000,食品饮料,1998-04-27
211,600197.SH,伊力特,2649348.000000,食品饮料,1999-09-16


In [4]:
idsty_stk_dic = {}
for index,row in df_stk_clean.iterrows():
    tmp_code = row['code']
    tmp_industry = row['industry']
    if tmp_industry not in idsty_stk_dic:
        idsty_stk_dic[tmp_industry] = [tmp_code]
    else:
        idsty_stk_dic[tmp_industry].append(tmp_code)
print(idsty_stk_dic.keys())
df_industry_count = pd.DataFrame(df_stk_clean['industry'].value_counts())
df_industry_count = df_industry_count.reset_index()
df_industry_count

dict_keys(['交通运输', '传媒', '公用事业', '农林牧渔', '医药生物', '商贸零售', '国防军工', '基础化工', '家用电器', '工业', '建筑材料', '建筑装饰', '房地产', '有色金属', '机械设备', '汽车', '煤炭', '环保', '电力设备', '电子', '石油石化', '社会服务', '美容护理', '计算机', '轻工制造', '通信', '钢铁', '银行', '非银金融', '食品饮料'])


,index,industry
0,医药生物,21
1,电子,21
2,电力设备,20
3,有色金属,12
4,非银金融,12
5,食品饮料,11
6,基础化工,11
7,计算机,10
8,家用电器,8
9,建筑装饰,8


# industry setting

In [5]:
target_industry = '交通运输'
df_stk_clean[df_stk_clean['industry']==target_industry]

,code,name,value,industry,IPO_DATE
0,601006.SH,大秦铁路,1675401.000000,交通运输,2006-08-01
1,000088.SZ,盐田港,127506.000000,交通运输,1997-07-28
2,600798.SH,宁波海运,1770720.000000,交通运输,1997-04-23
3,000099.SZ,中信海直,500916.000000,交通运输,2000-07-31


# return calculate

In [6]:
# # cbond_list = ['113516.SH','110043.SH',
# # ]
# cbond_list = idsty_stk_dic[target_industry]  
# # ###############################################################
# before_trade_date = '20201029'#监测当日前一个交易日
# present_date = '20201030' #监测当日
# # before_trade_date = '20201026'#监测当日前一个交易日
# # present_date = '20201027' #监测当日
# # next_trade_date = '20220917' #监测当日下一个自然日
# # ###############################################################

# start_date = pd.to_datetime(before_trade_date) 

# end_date = pd.to_datetime('20221030') #多一个月没关系，切片的时候，不需要这多出来的一个月的对子会切掉

# cbond_price = pd.DataFrame([])
# for code in cbond_list:
#     price_df = w.wsi(code, "close", start_date, end_date,usedf=True)[1]   #只要了收盘价
#     price_df.columns = [code]
#     cbond_price = pd.concat([cbond_price,price_df],axis=1)
# cbond_price = cbond_price.ffill()
# # cbond_price = cbond_price[cbond_price.index.time!=time(15,0,0)]
# # cbond_price = cbond_price.loc['2022-09-09 15:00:00':] #从前一个交易日的收盘价开始截取
# cbond_price_begin_time = before_trade_date[:4]+'-' + \
#                     before_trade_date[4:6]+'-' + \
#                     before_trade_date[6:8]+' 15:00:00'
# cbond_price = cbond_price.loc[cbond_price_begin_time:]
# cbond_price

In [7]:
read_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/price/{target_industry}.pkl'
cbond_price = pd.read_pickle(read_path)
# cbond_price = cbond_price[cbond_list]
cbond_price

,601006.SH,000088.SZ,600798.SH,000099.SZ
2020-10-29 15:00:00,6.43,6.33,3.04,7.85
2020-10-30 09:30:00,6.44,6.35,3.03,7.80
2020-10-30 09:31:00,6.45,6.35,3.04,7.80
2020-10-30 09:32:00,6.44,6.36,3.04,7.85
2020-10-30 09:33:00,6.42,6.35,3.04,7.83
...,...,...,...,...
2022-10-28 14:56:00,6.50,4.59,3.58,7.03
2022-10-28 14:57:00,6.50,4.59,3.58,7.03
2022-10-28 14:58:00,6.50,4.59,3.58,7.03
2022-10-28 14:59:00,6.50,4.59,3.58,7.03


In [8]:
# cbond_return = pd.DataFrame([])
# # tmp_stk_code = '601006.SH'
# # cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
# stock_preclose = cbond_price.iloc[0,:].tolist() # 初始时刻的stock前收盘价

# for index,row in cbond_price.iterrows():
    
#     # cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
#     for i in range(len(cbond_list)):
#         cbond_return.loc[index,cbond_list[i]] = row[cbond_list[i]]/stock_preclose[i] - 1
#         if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
#             # cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
#             stock_preclose[i] = row[cbond_list[i]]
        
# # 检查df是否有空值
# cbond_return.isnull().sum(axis=0)  # 按列统计
# cbond_return.isnull().sum(axis=1) # 按行统计
# cbond_return.isnull().sum(axis=1).sum() # 统计整张表

In [9]:
read_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/return/{target_industry}.pkl'
cbond_return = pd.read_pickle(read_path)
# cbond_return = cbond_return[cbond_list]
cbond_return

,601006.SH,000088.SZ,600798.SH,000099.SZ
2020-10-29 15:00:00,0.000000,0.000000,0.000000,0.000000
2020-10-30 09:30:00,0.001555,0.003160,-0.003289,-0.006369
2020-10-30 09:31:00,0.003110,0.003160,0.000000,-0.006369
2020-10-30 09:32:00,0.001555,0.004739,0.000000,0.000000
2020-10-30 09:33:00,-0.001555,0.003160,0.000000,-0.002548
...,...,...,...,...
2022-10-28 14:56:00,-0.019608,-0.031646,-0.037634,-0.051282
2022-10-28 14:57:00,-0.019608,-0.031646,-0.037634,-0.051282
2022-10-28 14:58:00,-0.019608,-0.031646,-0.037634,-0.051282
2022-10-28 14:59:00,-0.019608,-0.031646,-0.037634,-0.051282


In [10]:
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [11]:
cbond_return.isnull().sum(axis=0)  # 按列统计

601006.SH    0
000088.SZ    0
600798.SH    0
000099.SZ    0
dtype: int64

In [12]:
# cbond_return = cbond_return.dropna()

# correlation & cointegration test

In [13]:
# print(cbond_return.corr(method='pearson').describe())
pd.set_option('display.max_columns',None)
df_corr = cbond_return.corr(method='pearson')
for i in range(len(df_corr)):
    df_corr.iloc[i,i]=np.nan
tmp_se = df_corr.max()
# df_corr
print('max corr code:',f'\'{tmp_se.idxmax()}\'')
tmp_se.sort_values(ascending=False)

max corr code: '000088.SZ'


000088.SZ    0.521987
600798.SH    0.521987
000099.SZ    0.342891
601006.SH    0.259413
dtype: float64

In [16]:
main_code = '000088.SZ'
tmp_line = df_corr.loc[main_code]
tmp_line = pd.DataFrame(tmp_line)
for index,row in tmp_line.iterrows():
    if row[main_code] == tmp_se.loc[main_code]:
        # print(index)
        sec_code = index
        tmp_corr = tmp_se.loc[main_code]
print(f'\'{main_code}\',\'{sec_code}\'','@',tmp_corr)

'000088.SZ','600798.SH' @ 0.5219869863604332


In [18]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)
pair_code = [main_code,sec_code]
x_code = pair_code[0]
y_code = pair_code[1]   
x = np.log(cbond_price[x_code]).tolist()
y = np.log(cbond_price[y_code]).tolist()

cointegration_test(y, x),cointegration_test(x, y)

((-3.49205341095951,
  0.008193540132380087,
  59,
  117311,
  {'1%': -3.430405744500477,
   '5%': -2.8615646382359285,
   '10%': -2.566783114063855},
  -1024121.6965592767),
 (-3.4921902656536217,
  0.008189944540462976,
  59,
  117311,
  {'1%': -3.430405744500477,
   '5%': -2.8615646382359285,
   '10%': -2.566783114063855},
  -993548.5556303))

Connect to Sky fail. errCode:49


# output to pickle

In [ ]:
# save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/price/{target_industry}.pkl'
# cbond_price.to_pickle(save_path)
# save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/return/{target_industry}.pkl'
# cbond_return.to_pickle(save_path)